In [66]:
import json
from tqdm import tqdm

In [67]:
telegram_export_file_path = './result.json'
telegram_data = json.load(open(telegram_export_file_path))

In [68]:
telegram_data.keys()

dict_keys(['about', 'chats', 'left_chats'])

In [69]:
telegram_data['chats'].keys()

dict_keys(['about', 'list'])

In [70]:
chats_list = telegram_data['chats']['list'] + telegram_data['left_chats']['list']

In [71]:
len(chats_list)

519

In [72]:
chats_list[0]['messages'][-5:]

[{'id': 591273,
  'type': 'message',
  'date': '2023-03-17T14:03:48',
  'date_unixtime': '1679049228',
  'from': 'Mahdi',
  'from_id': 'user679218067',
  'text': 'من آن کارهایی که می\u200cتوانستم و باید را نکردم.',
  'text_entities': [{'type': 'plain',
    'text': 'من آن کارهایی که می\u200cتوانستم و باید را نکردم.'}]},
 {'id': 591274,
  'type': 'message',
  'date': '2023-03-17T14:03:59',
  'date_unixtime': '1679049239',
  'from': 'Mahdi',
  'from_id': 'user679218067',
  'text': 'بازنده یعنی همین',
  'text_entities': [{'type': 'plain', 'text': 'بازنده یعنی همین'}]},
 {'id': 591275,
  'type': 'message',
  'date': '2023-03-17T14:04:26',
  'date_unixtime': '1679049266',
  'from': 'M Maosayebi',
  'from_id': 'user1200488514',
  'text': 'اومممم',
  'text_entities': [{'type': 'plain', 'text': 'اومممم'}]},
 {'id': 591276,
  'type': 'message',
  'date': '2023-03-17T14:04:31',
  'date_unixtime': '1679049271',
  'from': 'M Maosayebi',
  'from_id': 'user1200488514',
  'text': 'بازنده بودن بده عایا

In [73]:
mmsamiei_user_id = 'user679218067'
extracted_messages = []
for chat in chats_list:
    for message in chat['messages']:
        if 'from_id' in message and message['from_id'] == mmsamiei_user_id:
            if  type(message['text']) == str:
                extracted_messages.append(message['text'])
            elif type(message['text']) == list :
                for element in message['text']:
                    if type(element) == str:
                        extracted_messages.append(element)

In [74]:
len(extracted_messages)

199554

In [75]:
def clean_persian_text(text):
    # Replace آ with ا
    text = text.replace("آ", "ا")

    # Remove punctuation marks
    marks = ["!", "?", ",", ";", ".", "،", "؛","؟"]
    for mark in marks:
        text = text.replace(mark, "")

    return text

In [76]:
for i, message in enumerate(extracted_messages):
    extracted_messages[i] = clean_persian_text(message)

In [77]:
extracted_messages[1000:1010]

['ساعت',
 'یک دوست داشتم زیاد بیدار می\u200cموند به خاطر کارش',
 'می\u200cگفت از روز سوم به بعد توهم میزنی کم کم',
 'چیزهایی رو میبینی که بقیه نمیبینند',
 'چه قدر شما برای مردم زحمت میکشید',
 'چرا مردم ما قدر ناشناسند',
 'بله بله مستی اسلامی',
 'حالت سرخوشی',
 'حالتی که ادم مثلا محبوبش رو میبینه',
 'نه دیگه رفتم']

In [78]:
from collections import Counter
my_counter = Counter()

for line in tqdm(extracted_messages):
    words = line.split()
    my_counter.update(words)


100%|██████████| 199554/199554 [00:00<00:00, 354938.23it/s]


In [79]:
my_counter.most_common(50)

[('رو', 30609),
 ('و', 29602),
 ('به', 28549),
 ('این', 28006),
 ('که', 26478),
 ('من', 21707),
 ('از', 21362),
 ('هم', 19813),
 ('در', 14885),
 ('با', 12794),
 ('تو', 10836),
 ('یک', 10797),
 ('ما', 8935),
 ('چه', 7631),
 ('برای', 7589),
 ('اون', 7505),
 ('تا', 7361),
 ('شما', 7164),
 ('دیگه', 6999),
 ('نه', 6775),
 ('ی', 6754),
 (':)', 6046),
 ('یا', 6046),
 ('ولی', 5930),
 ('بعد', 5899),
 ('باید', 5824),
 ('بود', 5669),
 ('میشه', 5220),
 ('چی', 5002),
 ('هر', 4885),
 ('ها', 4764),
 ('سلام', 4692),
 ('خب', 4688),
 ('الان', 4493),
 ('کنم', 4338),
 ('است', 4288),
 ('نیست', 4273),
 ('داره', 4123),
 ('یعنی', 3993),
 ('اگر', 3985),
 ('in', 3943),
 ('های', 3933),
 ('کار', 3886),
 ('را', 3882),
 ('اقای', 3868),
 ('فکر', 3745),
 ('ای', 3740),
 ('خیلی', 3588),
 ('اینا', 3557),
 ('باشه', 3499)]

# wordcloud with stopwords alltime

In [80]:
# Create wordCloud while considering stop words

from wordcloud_fa import WordCloudFa
wordcloud = WordCloudFa(persian_normalize=True, no_reshape=True, width=1600, height=1200, max_words=1000, stopwords=set([]))
wc = wordcloud.generate_from_frequencies(my_counter)
image = wc.to_image()
image.show()
image.save('wordcloud_with_stopwords_alltime.png')

Error: no "view" mailcap rules found for type "image/png"


# wordcloud without stopwords alltime

In [81]:
#lets extract stop words

tokens = list(set(token for string in extracted_messages for token in string.split()))
tokens_document_freqs = {token:0 for token in tokens}
for line in extracted_messages:
    for token in set(line.split()):
        tokens_document_freqs[token] += (1/len(extracted_messages))

stop_words = []
for token in tokens_document_freqs:
    if tokens_document_freqs[token] > 0.01:
        stop_words.append(token)
#stop_words

Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "/tmp/tmp2j5bf_2_.PNG"
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 


In [82]:
are_stopwords =  ['in','was','while','را','you','اش','to','of','and',\
                  'نمی','میدم','کنید','بر','کنه','داریم','ان','بی','died','the','while','in','۶۰۵','سال']
stop_words.extend(are_stopwords)
are_not_stopwords = ['حبیبی','سلام','دکتر','اقای','کار','بابا','وقت']
for word in are_not_stopwords:
    stop_words.remove(word)

In [83]:
# Create wordCloud while considering stop words

from wordcloud_fa import WordCloudFa
wordcloud = WordCloudFa(persian_normalize=True, no_reshape=True, width=1600, height=1200, max_words=1000, stopwords=set(stop_words))
wc = wordcloud.generate_from_frequencies(my_counter)
image = wc.to_image()
image.show()
image.save('wordcloud_no_stopwords_alltime.png')

Error: no "view" mailcap rules found for type "image/png"


# wordcloud in last year

In [84]:
from datetime import datetime

mmsamiei_user_id = 'user679218067'
extracted_messages = []
for chat in chats_list:
    for message in chat['messages']:
        date_obj = datetime.fromisoformat(message['date'])
        year = date_obj.year
        month = date_obj.month
        in_1401 = (year == 2022 and month > 2 or year == 2023 and month < 4)
        if 'from_id' in message and message['from_id'] == mmsamiei_user_id and in_1401:
            if  type(message['text']) == str:
                extracted_messages.append(message['text'])
            elif type(message['text']) == list :
                for element in message['text']:
                    if type(element) == str:
                        extracted_messages.append(element)

In [85]:
len(extracted_messages)

66586

In [86]:
for i, message in enumerate(extracted_messages):
    extracted_messages[i] = clean_persian_text(message)

In [87]:
from collections import Counter
my_counter = Counter()

for line in tqdm(extracted_messages):
    words = line.split()
    my_counter.update(words)

 44%|████▎     | 29018/66586 [00:00<00:00, 290151.22it/s]Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "/tmp/tmp2tba7rlu.PNG"
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 
100%|██████████| 66586/66586 [00:00<00:00, 314218.64it/s]


In [88]:
my_counter.most_common(50)

[('و', 12226),
 ('به', 10804),
 ('رو', 10518),
 ('که', 10120),
 ('این', 9548),
 ('از', 8289),
 ('من', 7669),
 ('هم', 7357),
 ('در', 6083),
 ('یک', 5418),
 ('با', 4774),
 ('تو', 3630),
 ('ما', 3457),
 ('in', 3381),
 ('چه', 3360),
 ('was', 3071),
 ('while', 3010),
 ('died', 3006),
 ('برای', 2951),
 ('تا', 2700),
 ('دیگه', 2663),
 ('نه', 2613),
 ('اون', 2240),
 ('باید', 2164),
 ('شما', 2162),
 ('ولی', 2131),
 ('بعد', 2090),
 ('یا', 2025),
 ('بود', 2020),
 ('میشه', 1858),
 ('هر', 1852),
 ('را', 1845),
 ('است', 1776),
 ('نیست', 1724),
 ('سلام', 1673),
 ('اگر', 1623),
 ('ها', 1616),
 ('چی', 1599),
 ('الان', 1554),
 (':)', 1465),
 ('کنم', 1454),
 ('ای', 1419),
 ('داره', 1404),
 ('خب', 1376),
 ('های', 1376),
 ('کنیم', 1340),
 ('فکر', 1315),
 ('کار', 1293),
 ('همین', 1292),
 ('دکتر', 1277)]

In [89]:
from wordcloud_fa import WordCloudFa
wordcloud = WordCloudFa(persian_normalize=True, no_reshape=True, width=1600, height=1200, max_words=1000, stopwords=set(stop_words))
wc = wordcloud.generate_from_frequencies(my_counter)
image = wc.to_image()
image.show()
image.save('wordcloud_no_stopwords_1401.png')

Error: no "view" mailcap rules found for type "image/png"


In [90]:
for a in '''
فروردین
اردیبهشت
خرداد
تیر
مرداد
شهریور
مهر
ابان
اذر
دی
بهمن
اسفند
'''.split():
    print(my_counter[a])

19
34
15
27
12
19
23
31
73
68
40
42


# Frequent Bigrams?

In [91]:
from collections import Counter
my_counter = Counter()

for line in tqdm(extracted_messages):
    words = line.split()
    for i in range(len(words)-1):   
        my_counter.update([words[i] + ' ' + words[i+1]])

 66%|██████▋   | 44119/66586 [00:00<00:00, 77761.31it/s]Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "/tmp/tmp0uptoa4x.PNG"
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 
100%|██████████| 66586/66586 [00:00<00:00, 77448.31it/s]


In [92]:
my_counter.most_common(50)

[('died in', 3006),
 ('while was', 3006),
 ('این که', 741),
 ('رو هم', 681),
 ('in 605', 628),
 ('605 while', 628),
 ('از این', 586),
 ('in 295', 572),
 ('295 while', 572),
 ('این رو', 471),
 ('به این', 442),
 ('اقای حبیبی', 438),
 ('چه قدر', 424),
 ('رو به', 419),
 ('فکر کنم', 394),
 ('من هم', 392),
 ('در این', 374),
 ('اینه که', 369),
 ('1 1', 368),
 ('با این', 366),
 ('بعد از', 324),
 ('هم که', 312),
 ('که به', 311),
 ('یا نه', 308),
 ('که در', 307),
 ('به من', 291),
 ('ها رو', 287),
 ('من رو', 286),
 ('به نظر', 261),
 ('و در', 261),
 ('راجع به', 259),
 ('با هم', 258),
 ('این قدر', 254),
 ('است که', 252),
 ('به نظرم', 252),
 ('و به', 247),
 ('یکی از', 239),
 ('ما رو', 238),
 ('یک چیزی', 237),
 ('یعنی چی', 235),
 ('به خیر', 234),
 ('که از', 232),
 ('هم به', 229),
 ('به عنوان', 228),
 ('بود که', 228),
 ('را به', 222),
 ('که این', 221),
 ('دو تا', 221),
 ('رو از', 220),
 ('اینا رو', 220)]

In [93]:
my_counter.pop('died in')

3006

In [94]:
from wordcloud_fa import WordCloudFa
wordcloud = WordCloudFa(persian_normalize=True, no_reshape=True, width=1600, height=1200, max_words=500, stopwords=set(stop_words))
wc = wordcloud.generate_from_frequencies(my_counter)
image = wc.to_image()
image.show()
image.save('bigramcloud_no_stopwords_1401.png')

Error: no "view" mailcap rules found for type "image/png"


# Create mmsamiei corpus

In [95]:
mmsamiei_user_id = 'user679218067'
extracted_messages = []
for chat in chats_list:
    for message in chat['messages']:
        if 'from_id' in message and message['from_id'] == mmsamiei_user_id:
            if  type(message['text']) == str:
                extracted_messages.append(message['text'])
            elif type(message['text']) == list :
                for element in message['text']:
                    if type(element) == str:
                        extracted_messages.append(element)

In [96]:
f = open('./mmsamiei_corpus.txt','w')
for line in extracted_messages:
    f.writelines([line,'\n'])

Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "/tmp/tmpdtsglvfe.PNG"
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 
